# CS 7180 Geometric Deep Learning, Homework 1: CNNs and GNNs

## TLDR

* **To start solving the homework, create a copy of this notebook:** `File -> Save a copy in Drive`.
* **To hand-in the homework**, email your PDF with answers to `biza.o@northeastern.edu` and add `biza.o@northeastern.edu` as a `Commentor` using the `Share` button **in your copy of this notebook**.
* Deadline is Wednesday February 1st, 5 PM EST.

## Solving the homework in Google Colab

[Google Colab](colab.research.google) is an online platform for running interactive Python session with [Jupyter](https://jupyter.org/) / [IPython](https://ipython.org/) (`.ipynb`) notebooks. We recommend you use it so that we can avoid problems with setting up local Python environment and so that we can use free GPUs to accelerate neural network training. 

If you have not used Python notebooks before, please check [this overview](https://colab.research.google.com/notebooks/basic_features_overview.ipynb). If you are new to Python, you can refer to [this excellent Python Numpy Tutorial](https://cs231n.github.io/python-numpy-tutorial/). We will be using [Numpy](https://numpy.org/) and [PyTorch](https://pytorch.org/) extensively to prepare data and train neural networks. For a PyTorch tutorial, follow [this link](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html) and select `Run in Google Colab`.

You cannot save the changes you made to this notebook. To start solving the homework, create a copy that you can edit: `File -> Save a copy in Drive`.

By default, your instance will run on a CPU. That is ok, but training neural network might take a while. You can (and should) request a free GPU instance using: `Runtime` -> `Change runtime type` -> `Hardware accelerator` -> `GPU` (-> `Delete previous runtime` -> `OK`). Note that your GPU instance will stop due to inactivity or a 12-hour time limit, but you can usually immediately request a new one. This will however delete all your progress (e.g. network weights) that is not saved in your code blocks, text blocks or Google Drive.

Another useful command to reset your instance (keeps your GPU):
`Runtime` -> `Restart runtime`.

## Handing in the homework

Please email a PDF with your answers to `biza.o@northeastern.edu`. We recommend you write-up your answers in latex (e.g. [overleaf.com](https://www.overleaf.com)). Please also share your copy of this notebook by clicking `Share` in the top-right corner, adding `biza.o@northeastern.edu`, setting me as `Commentor` and enabling `Notify people`. Another option is to email your `.ipynb` to me.

## Deadline
The deadline for this homework is **Wednesday February 1st, 5 PM EST**. You should email your solution to me by that time.

## (Optional) Solving the homework on your local machine
You can download this notebook using `Download -> Download .ipynb` and run it on your local machine. To set up a local environment, download [Python 3.6](https://www.python.org/) or later, set up a [virtual environment](https://docs.python.org/3/library/venv.html) and install [relevant packages](https://pip.pypa.io/en/stable/reference/requirements-file-format/) using [pip](https://pypi.org/project/pip/). On Linux, you would do something like this:

```
# Assuming you have Python >= 3.6 and python3-venv installed.
python3 -m venv venv
source ./venv/bin/activate
# Prevents opencv-python (which we might use) from taking forever to install.
pip install --upgrade pip setuptools wheel
pip install -r requirements.txt
pip install jupyter
# Run in the directory where this notebook is located. A browser window should open.
jupyter notebook .
```

You can run `!pip freeze` *in colab* to see the version of packages we are using. Then, you can copy the relevant packages (i.e. the ones we import) to your `requirements.txt` file or install them using `pip install x` in command line or `!pip install x` directly in the notebook. Importantly, to run on your GPU, you will need to install a `torch` version that matches your [CUDA](https://developer.nvidia.com/cuda-downloads) and [cuDNN](https://developer.nvidia.com/cudnn) versions. Please refer to the [PyTorch website](https://pytorch.org/). We might use the CPU version of tensorflow to prepare datasets.

# Python setup

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
import os
# I like to use typing, but you don't have to!
from typing import Any, Dict, Tuple, Union

import networkx as nx
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
from scipy.ndimage import shift
import scipy.sparse as sp
import tensorflow as tf
import tensorflow_datasets as tfds
import torch
from torch.utils.data import DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim
import torchvision
from torchvision import datasets, transforms
import torch.utils.data as data_utils

# We might use tensorflow to load datasets.
# Prevent tensorflow from stealing our GPU.
tf.config.experimental.set_visible_devices([], "GPU")

# Interactive plots.
%matplotlib inline

In [ ]:
# Runtime -> Change runtime type -> Hardware accelerator -> GPU (-> Delete previous runtime -> OK).
if torch.cuda.is_available() and torch.cuda.device_count() > 0:
  print("PyTorch running on a GPU. Use '.to(device)' to move tensors and models to GPU.")
  device = torch.device("cuda:0")
else:
  print("WARNING: PyTorch running on a CPU. See comment above to request a GPU instance.")
  device = torch.device("cpu")

# Part 1: Training a convolutional network on MNIST, measuring translation equivariance.

In this part, we will experiment with [MNIST](http://yann.lecun.com/exdb/mnist/), the quintessential deep learning dataset. You will implement and train a convolutional neural network, and measure its equivariance and invariance to translations. In 1999, [Yan LeCunn wrote](http://yann.lecun.com/exdb/publis/pdf/lecun-99.pdf) "A large degree of invariance to geometric transformations of the input can be achieved with this [referring to subsampling and pooling in a CNN] progressive reduction of spatial resolution compensated by progressive increase of the richness of the representation (the number of feature maps)." Let's see if that's true!

## Data preprocessing

In [ ]:
def prepare_mnist(batch_size: int) -> Tuple[DataLoader, DataLoader]:
  """Prepare MNIST training and evaluation data loaders in PyTorch.
  # https://github.com/pytorch/examples/blob/main/mnist/main.py
  """
  t = transforms.ToTensor()

  train_ds = datasets.MNIST("/tmp/mnist", download=True, train=True, transform=t)
  eval_ds = datasets.MNIST("/tmp/mnist", download=True, train=False, transform=t)

  train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
  eval_loader = DataLoader(eval_ds, batch_size=batch_size, shuffle=False, num_workers=1)

  return train_loader, eval_loader

In [ ]:
def translate(image: Union[torch.Tensor, NDArray], delta_x: float, delta_y: float) -> Union[torch.Tensor, NDArray]:
  """Translate an image. Either torch.Tensor or numpy.ndarray. Can be HxWxC or CxHxW."""
  is_torch = False
  if type(image) == torch.Tensor:
    # Pytorch to numpy.
    is_torch = True
    tmp = image.to("cpu").detach().numpy()
  else:
    tmp = image

  if len(tmp.shape) == 2:
    # HxW
    delta = [delta_x, delta_y]
  else:
    assert len(tmp.shape) == 3, "Invalid number of dimensions in an image, should be 2 or 3."
    if tmp.shape[0] in [1, 3]:
      # CxHxW
      delta = [0, delta_x, delta_y]
    elif tmp.shape[2] in [1, 3]:
      # HxWxC
      delta = [delta_x, delta_y, 0]
    else:
      raise ValueError("Invalid number of channels in an image, should be 1 or 3.")

  # Shift and automatically add zero padding.
  tmp = shift(image, delta)

  if is_torch:
    tmp = torch.tensor(tmp, dtype=image.dtype, device=image.device)
  return tmp

In [ ]:
train_loader, eval_loader = prepare_mnist(128)
example = next(iter(train_loader))
plt.figure(figsize=(10, 10))
for i in range(5*5):
  plt.subplot(5, 5, 1 + i)
  plt.title("Label: {:d}".format(example[1][i]))
  plt.imshow(example[0][i][0])
  plt.axis("off")

In [ ]:
t = transforms.ToTensor()
train_ds = datasets.MNIST("/tmp/mnist", download=True, train=True, transform=t)

In [ ]:
# train_ds_trans = torch.empty_like(train_ds[0][0])
# train_ds_trans = torch.tensor((6000,1,28,28), dtype=torch.float64)
# train_ds_trans.new_zeros((6000,1,28,28)).shape
train_ds_ = torch.FloatTensor(1, 1, 28, 28) 
torch.unsqueeze(translate(train_ds[i][0], -1, -3),0).shape

In [ ]:
img = example[0][0]
transl = [-5, -2, 0, 2, 5]
print("Translate by {}.".format(transl))
plt.figure(figsize=(10, 10))
for i in range(5):
  for j in range(5):
    tmp = translate(img, transl[i], transl[j])
    plt.subplot(5, 5, 1 + j + i * 5)
    plt.imshow(tmp[0])
    plt.axis("off")

## Training a baseline

In [ ]:
class MLPBaseline(nn.Module):
  """A Multi Layer Perceptron baseline.
  It flattens the input image and applies three fully-connected layers
  each followed by an activation."""
  def __init__(self):
    super().__init__()

    self.fc1 = nn.Linear(28 * 28, 128)
    self.fc2 = nn.Linear(128, 128)
    self.fc3 = nn.Linear(128, 10)
    self.act = nn.ReLU()

  def forward(self, x: torch.Tensor) -> torch.Tensor:

    # print("shape1=",x.shape)
    x = x.reshape(x.shape[0], -1)
    # print("shape2=",x.shape)

    x = self.act(self.fc1(x))
    # print("shape3=",x.shape)
    x = self.act(self.fc2(x))
    # print("shape4=",x.shape)
    # Our loss function expect log probabilities of the 10 digit classes.
    x = F.log_softmax(self.fc3(x), dim=-1)
    # print("shape5=",x.shape)

    return x

In [ ]:
def train(train_loader: DataLoader, model: nn.Module,
          optimizer: optim.Optimizer, epochs: int):

  losses = []
  # 1 epoch = 1 pass through the dataset.
  for epoch in range(epochs):
    print("Epoch {:d} / {:d}".format(epoch, epochs))
    # train_loader steps once it iterates over all data.
    for train_step, batch in enumerate(train_loader):
      # train_loader gives us batched images (CxHxW) and labels (integers).
      # If we are using a GPU we need to move them to the device.
      inputs, labels = batch[0].to(device), batch[1].to(device)
      # print(inputs.size())
      # input size = torch.Size([128, 1, 28, 28])

      # Reset last optimization step.
      optimizer.zero_grad()
      # Make prediction, calculate loss.
      outputs = model(inputs)
      loss = F.nll_loss(outputs, labels)
      # Compute gradients and update weights.
      loss.backward()
      optimizer.step()
      losses.append(loss.item())

      if train_step > 0 and train_step % 50 == 0:
        print("Mean of last 50 losses: {:f}".format(np.mean(losses[-50:])))
  # plt.plot(losses)
  # plt.xlabel("Training step")
  # plt.ylabel("Loss")
  return losses


In [ ]:
# We don't need gradients during evaluation.
@torch.no_grad()
def evaluate(eval_loader: DataLoader, model: nn.Module):

  accuracy = []

  for batch in eval_loader:
    inputs, labels = batch[0].to(device), batch[1].to(device)
    outputs = model(inputs)
    pred_labels = torch.argmax(outputs, dim=-1)
    tmp = (labels == pred_labels).float().mean()
    accuracy.append(tmp.item())

  # We compute the mean of means over batches.
  # This could be slightly skewed if the last batch is smaller.
  # Does not matter too much here.
  accuracy = np.mean(accuracy)
  return accuracy
  print("Correct answer in {:.1f}% of cases.".format(accuracy * 100))

In [ ]:
@torch.no_grad()
def plot_predictions(eval_loader: DataLoader, model: nn.Module):
  example = next(iter(eval_loader))
  inputs = example[0].to(device)
  outputs = model(inputs)
  pred_labels = torch.argmax(outputs, dim=-1).to("cpu").numpy()

  plt.figure(figsize=(10, 10))
  for i in range(5*5):
    plt.subplot(5, 5, 1 + i)
    plt.title("Label: {:d}".format(pred_labels[i]))
    plt.imshow(example[0][i][0])
    plt.axis("off")

In [ ]:
model = MLPBaseline().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
train(train_loader, model, optimizer, 5)

In [ ]:
evaluate(eval_loader, model)

In [ ]:
plot_predictions(eval_loader, model)

## == Task 1 ==

[**Question 1**] Implement a convolutional neural network and reports its learning curve (loss vs training steps) and validation accuracy compared to the MLP baseline we implemented. An *example* of a model you could use is: `CNN(64 kernels, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)) -> ReLU -> CNN(64 kernels, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)) -> ReLU -> CNN(64 kernels, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)) -> ReLU -> Flatten to vector -> Linear(128) -> ReLU -> Linear(10) -> LogSoftmax.`

[**Question 2**] Plot validation accuracy vs training dataset size for the CNN and the MLP. The full MNIST training set size is 60k. You can choose any interesting range of dataset sizes. Make sure you actually limit the dataset size: [hint](https://stackoverflow.com/questions/44856691/how-do-you-alter-the-size-of-a-pytorch-dataset).

[**Question 3**] Measure the **invariance** of the CNN and MLP predictions to translations of the input image. You can do so by randomly translating the input image (with the limit that the digit is still somewhat within the image) and measuring the change of the predicted probabilities compared to the image of a centered digit. Repeat the same measurement for untrained (randomly initialized) networks. It is ok to perform this and all following experiments with only a few validation images. Discuss the results.

[**Question 4**] Add several max pooling or average pooling layers to your convolutional network (you can set convolutional strides to 1 to avoid downsampling too quickly). Does pooling make the network more invariant?

[**Question 5**] Remove any stride and max pooling from your network, so that the resolution of your feature map is always 28 * 28 (the original image size). The final prediction layer can first flatten the feature map, or maybe perform average pooling, followed by some fully-connected layers. Measure the translation **equivariance** of any two intermediate feature maps in your convolutional network. You want to translate the input image and see if the feature map (of the same size) translates accordingly. Repeat the same measurement for an untrained (randomly initialized) network.

# **=========================== Task1 ===========================**

[Question 1] Implement a convolutional neural network and reports its learning curve (loss vs training steps) and validation accuracy compared to the MLP baseline we implemented. An example of a model you could use is: 
CNN(64 kernels, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)) -> ReLU -> CNN(64 kernels, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)) -> ReLU -> CNN(64 kernels, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2)) -> ReLU -> Flatten to vector -> Linear(128) -> ReLU -> Linear(10) -> LogSoftmax.

In [ ]:
class CNN(nn.Module):
  """A Multi Layer Perceptron baseline.
  It flattens the input image and applies three fully-connected layers
  each followed by an activation."""
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,64,5,2,2)
    self.conv2 = nn.Conv2d(64,64,5,2,2)
    self.conv3 = nn.Conv2d(64,64,5,2,2)
    

    self.fc1 = nn.Linear(64*4*4,128)
    self.fc2 = nn.Linear(128,10)
    # self.fc3 = nn.Linear(128,10)


  def forward(self, x: torch.Tensor) -> torch.Tensor:
    # x = x.reshape(x.shape[0], -1)
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = F.relu(self.conv3(x))
    # print("x1",x.shape)
    x = torch.flatten(x,1)
    x = F.relu(self.fc1(x)) 
    # x = F.relu(self.fc2(x))
    x = F.log_softmax(self.fc2(x),dim = -1)

    return x

In [ ]:
t = transforms.ToTensor()
tr = datasets.MNIST('../data', train=True, download=True, transform=t)
te = datasets.MNIST('../data', train=False, transform=t)

In [ ]:
def createList(r1, r2):
    return [item for item in range(r1, r2+1) if item%1000==0]
split_list = createList(500,30000)


In [ ]:
# tr_split_len = 10000
# te_split_len = 1000
acclistMLP = []
acclistCNN = []

for split_len in split_list:
  part_tr = torch.utils.data.random_split(tr, [split_len, len(tr)-split_len])[0]
  part_te = torch.utils.data.random_split(te, [split_len, len(te)-split_len])[0]

  train_set = DataLoader(part_tr, batch_size=128, shuffle=True, num_workers=2)
  eval_set = DataLoader(part_te, batch_size=128, shuffle=True, num_workers=1)

  modelMLP = MLPBaseline().to(device)
  optimizer = optim.Adam(modelMLP.parameters(), lr=0.001)
  # lossMLP = train(train_loader, modelMLP, optimizer, 5)
  lossMLP = train(train_set, modelMLP, optimizer, 5)

  modelCNN = CNN().to(device)
  optimizer = optim.Adam(modelCNN.parameters(), lr=0.001)
  # lossCNN = train(train_loader, modelCNN, optimizer, 5)
  lossCNN = train(train_set, modelCNN, optimizer, 5)

  acclistMLP.append(evaluate(eval_set, modelMLP))
  acclistCNN.append(evaluate(eval_set, modelCNN))


plt.plot(split_list,acclistMLP)
plt.plot(split_list,acclistCNN)
plt.xlabel("Training dataset len")
plt.ylabel("Validation Accuracy")

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(split_list,acclistMLP,label="MLP")
plt.plot(split_list,acclistCNN,label="CNN")
plt.legend(loc="upper left")
plt.xlabel("Training dataset size")
plt.ylabel("Validation Accuracy")

In [ ]:
modelMLP = MLPBaseline().to(device)
optimizer = optim.Adam(modelMLP.parameters(), lr=0.001)
# lossMLP = train(train_loader, modelMLP, optimizer, 5)
lossMLP = train(train_loader, modelMLP, optimizer, 5)

modelCNN = CNN().to(device)
optimizer = optim.Adam(modelCNN.parameters(), lr=0.001)
# lossCNN = train(train_loader, modelCNN, optimizer, 5)
lossCNN = train(train_loader, modelCNN, optimizer, 5)

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(lossCNN, label="CNN")
plt.plot(lossMLP, label="MLPBaseline")
plt.legend(loc="upper right")
plt.xlabel("Training step")
plt.ylabel("Loss")

In [ ]:
evaluate(eval_loader, modelMLP)
evaluate(eval_loader, modelCNN)

[Question 3] Measure the invariance of the CNN and MLP predictions to translations of the input image. You can do so by randomly translating the input image (with the limit that the digit is still somewhat within the image) and measuring the change of the predicted probabilities compared to the image of a centered digit. Repeat the same measurement for untrained (randomly initialized) networks. It is ok to perform this and all following experiments with only a few validation images. Discuss the results.

In [ ]:
import random
transl = [-5, -2, 0, 2, 5]
train_ds_trans = torch.unsqueeze(translate(train_ds[i][0], -1, -3), 0)
for i in range(10):
  x_trans = random.randint(0, 4)
  y_trans = random.randint(0, 4)
  new = torch.unsqueeze(translate(train_ds[i][0], transl[x_trans], transl[y_trans]),0)
  train_ds_trans = torch.cat([train_ds_trans,new],0)


In [ ]:
train_loader, eval_loader = prepare_mnist(128)
example = next(iter(train_loader))
plt.figure(figsize=(10, 10))
for i in range(5*5):
  plt.subplot(5, 5, 1 + i)
  plt.title("Label: {:d}".format(example[1][i]))
  plt.imshow(example[0][i][0])
  plt.axis("off")

In [ ]:
img = example[0][13]
translated_image_eight = torch.unsqueeze(img, 0)
transl = [-3, -2, 0, 2, 3]
print("Translate by {}.".format(transl))
plt.figure(figsize=(10, 10))
for i in range(5):
  for j in range(5):
    tmp = torch.unsqueeze(translate(img, transl[i], transl[j]),0)
    translated_image_eight = torch.cat([translated_image_eight,tmp],0)
    plt.subplot(5, 5, 1 + j + i * 5)
    plt.imshow(tmp[0][0])
    plt.axis("off")

In [ ]:
translated_image_eight_temp = torch.unsqueeze(translated_image_eight,0)
eval_loader = DataLoader(translated_image_eight_temp, batch_size=12, shuffle=False, num_workers=1)

accuracyMLP = []
accuracyCNN = []
accuracyCNNwithMaxPooling = []
labels = 4
for batch in eval_loader:
  inputs = batch[0].to(device)
  print(inputs.shape)
  outputs = modelMLP(inputs)
  pred_labels = torch.argmax(outputs, dim=-1)
  tmp = (labels == pred_labels).float().mean()
  accuracyMLP.append(tmp.item())
  outputs = modelCNN(inputs)
  pred_labels = torch.argmax(outputs, dim=-1)
  tmp = (labels == pred_labels).float().mean()
  accuracyCNN.append(tmp.item())
  outputs = modelCNNwithMaxPooling(inputs)
  pred_labels = torch.argmax(outputs, dim=-1)
  tmp = (labels == pred_labels).float().mean()
  accuracyCNNwithMaxPooling.append(tmp.item())

# We compute the mean of means over batches.
# This could be slightly skewed if the last batch is smaller.
# Does not matter too much here.
accuracyMLP = np.mean(accuracyMLP)
accuracyCNN = np.mean(accuracyCNN)
accuracyCNNwithMaxPooling = np.mean(accuracyCNNwithMaxPooling)

print("Correct answer in {:.1f}% of cases by using MLP network.".format(accuracyMLP * 100))
print("Correct answer in {:.1f}% of cases by using CNN network.".format(accuracyCNN * 100))
print("Correct answer in {:.1f}% of cases by using CNN network with max pooling.".format(accuracyCNNwithMaxPooling * 100))
# evaluate(eval_loader, modelCNN)

In [ ]:
modelMLPuntrained = MLPBaseline().to(device)
# optimizer = optim.Adam(modelMLPuntrained.parameters(), lr=0.001)
# lossMLP = train(train_10k, modelMLPuntrained, optimizer, 5)

modelCNNuntrained = CNN().to(device)
# optimizer = optim.Adam(modelCNNuntrained.parameters(), lr=0.001)
# lossCNN = train(train_10k, modelCNNuntrained, optimizer, 5)

In [ ]:
accuracyMLP = []
accuracyCNN = []
labels = 8
for batch in eval_loader:
  inputs = batch[0].to(device)
  print(inputs.shape)
  outputs = modelMLPuntrained(inputs)
  pred_labels = torch.argmax(outputs, dim=-1)
  tmp = (labels == pred_labels).float().mean()
  accuracyMLP.append(tmp.item())
  outputs = modelCNNuntrained(inputs)
  pred_labels = torch.argmax(outputs, dim=-1)
  tmp = (labels == pred_labels).float().mean()
  accuracyCNN.append(tmp.item())

# We compute the mean of means over batches.
# This could be slightly skewed if the last batch is smaller.
# Does not matter too much here.
accuracyMLP = np.mean(accuracyMLP)
accuracyCNN = np.mean(accuracyCNN)
print("Correct answer in {:.1f}% of cases by using untrained MLP network.".format(accuracyMLP * 100))
print("Correct answer in {:.1f}% of cases by using untrained CNN network.".format(accuracyCNN * 100))

[Question 4] Add several max pooling or average pooling layers to your convolutional network (you can set convolutional strides to 1 to avoid downsampling too quickly). Does pooling make the network more invariant?

In [ ]:
class CNNwithMaxPooling(nn.Module):
  """A Multi Layer Perceptron baseline.
  It flattens the input image and applies three fully-connected layers
  each followed by an activation."""
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,64,5,1,2)
    self.pool1 = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(64,64,5,1,2)
    self.pool2 = nn.MaxPool2d(2,2)
    self.conv3 = nn.Conv2d(64,64,5,1,2)
    self.pool3 = nn.MaxPool2d(2,2)
    self.fc1 = nn.Linear(64*3*3,128)
    self.fc2 = nn.Linear(128,10)
    # self.fc3 = nn.Linear(128,10)


  def forward(self, x: torch.Tensor) -> torch.Tensor:
    # x = x.reshape(x.shape[0], -1)
    x = F.relu(self.conv1(x))
    x = self.pool1(x)
    x = F.relu(self.conv2(x))
    x = self.pool2(x)
    x = F.relu(self.conv3(x))
    x = self.pool3(x)
    # print("x1",x.shape)
    x = torch.flatten(x,1)
    x = F.relu(self.fc1(x)) 
    # x = F.relu(self.fc2(x))
    x = F.log_softmax(self.fc2(x),dim = -1)

    return x


In [ ]:
modelCNNwithMaxPooling = CNNwithMaxPooling().to(device)
optimizer = optim.Adam(modelCNNwithMaxPooling.parameters(), lr=0.001)
# lossCNN = train(train_loader, modelCNN, optimizer, 5)
lossmodelCNNwithMaxPooling = train(train_loader, modelCNNwithMaxPooling, optimizer, 5)

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(lossCNN, label="CNN")
plt.plot(lossmodelCNNwithMaxPooling, label="CNN with MaxPooling")
plt.plot(lossMLP, label="MLPBaseline")

plt.legend(loc="upper right")
plt.xlabel("Training step")
plt.ylabel("Loss")

In [ ]:
# t = transforms.ToTensor()
# eval_ds = datasets.MNIST("/tmp/mnist", download=True, train=False, transform=t)
# eval_loader = DataLoader(eval_ds, batch_size=128, shuffle=False, num_workers=1)
accuracyMLP = []
accuracyCNN = []
accuracyCNNwithMaxPooling = []
labels = 5
for batch in eval_loader:
  inputs = batch[0].to(device)
  print(inputs.shape)
  outputs = modelMLP(inputs)
  pred_labels = torch.argmax(outputs, dim=-1)
  tmp = (labels == pred_labels).float().mean()
  accuracyMLP.append(tmp.item())
  outputs = modelCNN(inputs)
  pred_labels = torch.argmax(outputs, dim=-1)
  tmp = (labels == pred_labels).float().mean()
  accuracyCNN.append(tmp.item())
  outputs = modelCNNwithMaxPooling(inputs)
  pred_labels = torch.argmax(outputs, dim=-1)
  tmp = (labels == pred_labels).float().mean()
  accuracyCNNwithMaxPooling.append(tmp.item())

# We compute the mean of means over batches.
# This could be slightly skewed if the last batch is smaller.
# Does not matter too much here.
accuracyMLP = np.mean(accuracyMLP)
accuracyCNN = np.mean(accuracyCNN)
accuracyCNNwithMaxPooling = np.mean(accuracyCNNwithMaxPooling)

print("Correct answer in {:.1f}% of cases by using MLP network.".format(accuracyMLP * 100))
print("Correct answer in {:.1f}% of cases by using CNN network.".format(accuracyCNN * 100))
print("Correct answer in {:.1f}% of cases by using CNN network with max pooling.".format(accuracyCNNwithMaxPooling * 100))
# evaluate(eval_loader, modelCNN)

[Question 5] Remove any stride and max pooling from your network, so that the resolution of your feature map is always 28 * 28 (the original image size). The final prediction layer can first flatten the feature map, or maybe perform average pooling, followed by some fully-connected layers. Measure the translation equivariance of any two intermediate feature maps in your convolutional network. You want to translate the input image and see if the feature map (of the same size) translates accordingly. Repeat the same measurement for an untrained (randomly initialized) network.

In [ ]:
class CNNnoStridePooling(nn.Module):
  """A Multi Layer Perceptron baseline.
  It flattens the input image and applies three fully-connected layers
  each followed by an activation."""
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,64,5,1,2)
    self.conv2 = nn.Conv2d(64,64,5,1,2)
    self.conv3 = nn.Conv2d(64,64,5,1,2)
    self.fc1 = nn.Linear(64*28*28,128)
    self.fc2 = nn.Linear(128,10)
    self.fm1 = None
    self.fm2 = None
    self.fm3 = None
    


  def forward(self, x: torch.Tensor) -> torch.Tensor:
    # x = x.reshape(x.shape[0], -1)
    x = F.relu(self.conv1(x))
    self.fm1 = x
    x = F.relu(self.conv2(x))
    self.fm2 = x
    x = F.relu(self.conv3(x))
    self.fm3 = x
    # print("x1",x.shape)
    x = torch.flatten(x,1)
    x = F.relu(self.fc1(x)) 
    # x = F.relu(self.fc2(x))
    x = F.log_softmax(self.fc2(x),dim = -1)

    return x


In [ ]:
modelCNNnoStridePooling = CNNnoStridePooling().to(device)
# optimizer = optim.Adam(modelCNNnoStridePooling.parameters(), lr=0.001)
# # lossCNN = train(train_loader, modelCNN, optimizer, 5)
# lossmodelCNNnoStridePooling = train(train_loader, modelCNNnoStridePooling, optimizer, 5)


In [ ]:
plt.imshow(modelCNNnoStridePooling.fm2.cpu().detach().numpy()[2][0])

In [ ]:
img = example[0][0]
translated_image_eight = torch.unsqueeze(img, 0)
transl = [-3, -2, 0, 2, 3]
print("Translate by {}.".format(transl))
plt.figure(figsize=(10, 10))
for i in range(5):
  for j in range(5):
    tmp = torch.unsqueeze(translate(img, transl[i], transl[j]),0)
    translated_image_eight = torch.cat([translated_image_eight,tmp],0)
    plt.subplot(5, 5, 1 + j + i * 5)
    plt.imshow(tmp[0][0])
    plt.axis("off")

In [ ]:
eval_loader

In [ ]:
translated_image_eight_temp = torch.unsqueeze(translated_image_eight,0)
eval_loader = DataLoader(translated_image_eight_temp, batch_size=1, shuffle=False, num_workers=1)

accuracyMLP = []
accuracyCNN = []
accuracyCNNwithMaxPooling = []
accuracyCNNnoStridePooling = []
labels = 3
plt.figure(figsize=(10, 10))
i = 0
j = 0
count = -25
for batch in eval_loader:
  inputs = batch[0].to(device)
  for i in range(25):
    i = i+1
    if i % 5==5:
      j = j+1
      i = 0
    outputs = modelCNNnoStridePooling(inputs)
    pred_labels = torch.argmax(outputs, dim=-1)
    tmp = (labels == pred_labels).float().mean()
    plt.subplot(5, 5, i + j * 5)
    plt.imshow(modelCNNnoStridePooling.fm1.cpu().detach().numpy()[count][10])
    plt.axis("off")
    count = count + 1

In [ ]:
labels = 3
plt.figure(figsize=(10, 10))
i = 0
j = 0
count = -25
for batch in eval_loader:
  inputs = batch[0].to(device)
  for i in range(25):
    i = i+1
    if i % 5==5:
      j = j+1
      i = 0
    outputs = modelCNNnoStridePooling(inputs)
    pred_labels = torch.argmax(outputs, dim=-1)
    tmp = (labels == pred_labels).float().mean()
    plt.subplot(5, 5, i + j * 5)
    plt.imshow(modelCNNnoStridePooling.fm2.cpu().detach().numpy()[count][10])
    plt.axis("off")
    count = count + 1

In [ ]:
labels = 3
plt.figure(figsize=(10, 10))
i = 0
j = 0
count = -25
for batch in eval_loader:
  inputs = batch[0].to(device)
  for i in range(25):
    i = i+1
    if i % 5==5:
      j = j+1
      i = 0
    outputs = modelCNNnoStridePooling(inputs)
    pred_labels = torch.argmax(outputs, dim=-1)
    tmp = (labels == pred_labels).float().mean()
    plt.subplot(5, 5, i + j * 5)
    plt.imshow(modelCNNnoStridePooling.fm3.cpu().detach().numpy()[count][10])
    plt.axis("off")
    count = count + 1

In [ ]:
# plt.imshow(modelCNNnoStridePooling.fm1.cpu().detach().numpy()[-8][10])
# conv(T*Img)
img_fm1 = modelCNNnoStridePooling.fm1.cpu().detach().numpy()[-13][10]
img_fm2 = modelCNNnoStridePooling.fm2.cpu().detach().numpy()[-13][10]
img_fm3 = modelCNNnoStridePooling.fm3.cpu().detach().numpy()[-13][10]
plt.imshow(img_fm1)
# tmp = torch.unsqueeze(translate(img, transl[i], transl[j]),0)
# translated_image_eight = torch.cat([translated_image_eight,tmp],0)
# plt.subplot(5, 5, 1 + j + i * 5)
# plt.imshow(tmp[0][0])

In [ ]:
img_t_fm1 = modelCNNnoStridePooling.fm1.cpu().detach().numpy()[-1][10]
img_t_fm2 = modelCNNnoStridePooling.fm2.cpu().detach().numpy()[-1][10]
img_t_fm3 = modelCNNnoStridePooling.fm3.cpu().detach().numpy()[-1][10]

# T^-1*[conv(T*Img)]
img_t_fm1_t_ = translate(img_t_fm1, -3, -3)
img_t_fm2_t_ = translate(img_t_fm2, -3, -3)
img_t_fm3_t_ = translate(img_t_fm3, -3, -3)
plt.imshow(img_t_fm1_t_)

In [ ]:

from scipy.linalg import norm
from scipy import sum, average
def normalize(arr):
    rng = arr.max()-arr.min()
    amin = arr.min()
    return (arr-amin)*255/rng
def compare_images(img1, img2):
    # normalize to compensate for exposure difference, this may be unnecessary
    # consider disabling it
    img1 = normalize(img1)
    img2 = normalize(img2)
    # calculate the difference and its norms
    diff = img1 - img2  # elementwise for scipy arrays
    m_norm = sum(abs(diff))  # Manhattan norm
    z_norm = norm(diff.ravel(), 0)  # Zero norm
    return (m_norm, z_norm)

In [ ]:
compare_images(img_t_fm3_t_,img_fm3)

In [ ]:
img
img_t = translate(img, 2, 3)
plt.imshow(img_t[0])

In [ ]:
modelMLPuntrained = MLPBaseline().to(device)

modelCNNuntrained = CNN().to(device)

modelCNNnoSPuntrained = CNNnoStridePooling().to(device)

In [ ]:
accuracyMLP = []
accuracyCNN = []
accuracyCNNnoStridePooling = []
labels = 5
for batch in eval_loader:
  inputs = batch[0].to(device)

  outputs = modelMLPuntrained(inputs)
  pred_labels = torch.argmax(outputs, dim=-1)
  tmp = (labels == pred_labels).float().mean()
  accuracyMLP.append(tmp.item())

  outputs = modelCNNuntrained(inputs)
  pred_labels = torch.argmax(outputs, dim=-1)
  tmp = (labels == pred_labels).float().mean()
  accuracyCNN.append(tmp.item())

  outputs = modelCNNnoSPuntrained(inputs)
  pred_labels = torch.argmax(outputs, dim=-1)
  tmp = (labels == pred_labels).float().mean()
  accuracyCNNnoStridePooling.append(tmp.item())

# We compute the mean of means over batches.
# This could be slightly skewed if the last batch is smaller.
# Does not matter too much here.
accuracyMLP = np.mean(accuracyMLP)
accuracyCNN = np.mean(accuracyCNN)
accuracyCNNnoStridePooling = np.mean(accuracyCNNnoStridePooling)
print("Correct answer in {:.1f}% of cases by using untrained MLP network.".format(accuracyMLP * 100))
print("Correct answer in {:.1f}% of cases by using untrained CNN network.".format(accuracyCNN * 100))
print("Correct answer in {:.1f}% of cases by using untrained CNN network without stride and maxpolling.".format(accuracyCNNnoStridePooling * 100))



# Part 2: Training a Graph Convolutional Network on a citation graph dataset

We will be training a [Graph Convolutional Network](https://arxiv.org/abs/1609.02907), perhaps the most used graph neural network model, on a dataset of citations. The neural network will learn to classify an academic paper into several categories based on its references (edges in a graph) and other features. As the story goes, the GNC paper was developed and written in three weeks. You only have two.

## Downloading a dataset and preprocesing data

Cora was an ancient proof-of-concept search engine for computer science papers [1]. It is also the name of a citation graph dataset [2, 3]. The dataset contains papers as nodes and references between papers as edges. Each paper is represented as a binary vector with 1433 bits. Each bit encodes the presence or absence of a certain word in the paper (bag of words). Bonus points if you can find what specific words these bits encode ... I couldn't. Papers are sorted into seven classes, which I believe correspond to major categories in CS: AI, Hardware, Security, etc. But, most papers don't have a label. Our goal is to learn to classify papers based on their bag of words feature vectors, their neighbors and their training class. Then, we should be able to generalize to other papers, which are labeled for the prupose of validation.

[1] http://www.kamalnigam.com/papers/cora-aaaiss99.pdf

[2] https://relational.fit.cvut.cz/dataset/CORA

[3] http://eliassi.org/papers/ai-mag-tr08.pdf

In [ ]:
# Download cora dataset node and edge data from github.
!wget -P /tmp/cora https://github.com/tkipf/pygcn/raw/master/data/cora/cora.cites
!wget -P /tmp/cora https://github.com/tkipf/pygcn/raw/master/data/cora/cora.content

In [ ]:
# You should have cora.cites and cora.content.
# If you run the download code several times you will get duplicates, which is ok.
!ls /tmp/cora

In [ ]:
# https://github.com/tkipf/pygcn/blob/master/pygcn/utils.py
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot


def load_data(path="/tmp/cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

Below, you have:
* features: nodes times number of words in vocabulary,
* labels: node labels, zero for unlabeled nodes,
* idx_train: indices of nodes that have training labels,
* idx_val: indices of nodes that have validation labels,
* idx_test: indices of nodes that have testing labels,
* adj: a **sparse tensor** representing an adjacency matrix, adj[i, j]: indices if paper i cites paper j.

How to work with a [sparse torch tensor](https://pytorch.org/docs/stable/sparse.html)?
* You can call **adj.to_dense()** to convert it to a standard torch tensor. This is probably the best approach, unless your Graph Convolution model is too slow.
* Call **torch.sparse.mm** to multiply a sparse tensor (first argument) with a normal tensor (second argument).

In [ ]:
adj, features, labels, idx_train, idx_val, idx_test = load_data()

In [ ]:
features[idx_val].shape

In [ ]:
features.shape

In [ ]:
adj.shape

## Training a baseline

Below is a simple proof-of-concept baseline that classifies papers solely on their bag of words vectors. It achieves a validation accuracy of around 54%, which is somewhat better than only predicting class 5 with 46% accuracy. Your task is to use the adjacency matrix (adj) with a Graph Convolution to improve this model.

In [ ]:
class MLPBaseline(nn.Module):

  def __init__(self):
    super().__init__()

    self.fc1 = nn.Linear(1433, 128)
    self.fc2 = nn.Linear(128, 128)
    self.fc3 = nn.Linear(128, 7)
    self.act = nn.ReLU()

  def forward(self, x: torch.Tensor) -> torch.Tensor:

    x = self.act(self.fc1(x))
    x = self.act(self.fc2(x))
    x = F.log_softmax(self.fc3(x), dim=-1)

    return x

In [ ]:
def train(features: torch.Tensor, labels: torch.Tensor, idx_train: torch.Tensor,
          model: nn.Module, optimizer: optim.Optimizer, epochs: int):
  
  features = features[idx_train].to(device)
  labels = labels[idx_train].to(device)

  losses = []
  # 1 epoch = 1 pass through the dataset.

  for epoch in range(epochs):
    # Reset last optimization step.
    optimizer.zero_grad()
    # Make prediction, calculate loss.
    outputs = model(features)
    loss = F.nll_loss(outputs, labels)
    # Compute gradients and update weights.
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

    if epoch > 0 and epoch % 50 == 0:
      print("Epoch {:d} / {:d}".format(epoch, epochs))
      print("Mean of last 50 losses: {:f}".format(np.mean(losses[-50:])))

  plt.plot(losses)
  plt.xlabel("Training step")
  plt.ylabel("Loss")
  return losses

In [ ]:
# We don't need gradients during evaluation.
@torch.no_grad()
def evaluate(features: torch.Tensor, labels: torch.Tensor,
             idx_val: torch.Tensor, model: nn.Module):

  features = features[idx_val].to(device)
  labels = labels[idx_val].to(device)

  accuracy = []

  outputs = model(features)
  pred_labels = torch.argmax(outputs, dim=-1)
  tmp = (labels == pred_labels).float().mean()
  accuracy.append(tmp.item())

  accuracy = np.mean(accuracy)
  return accuracy
  print("Correct answer in {:.1f}% of cases.".format(accuracy * 100))

In [ ]:
features.shape

In [ ]:
modelMLP = MLPBaseline().to(device)
optimizer = optim.Adam(modelMLP.parameters(), lr=0.001)
MLP_losses = train(features, labels, idx_train, modelMLP, optimizer, 300)
evaluate(features, labels, idx_val, modelMLP)

## == Task 2 ==

[**Question 1**] Implement a Graph Convolutional Network (https://arxiv.org/abs/1609.02907) and train it on the Cora dataset. The network should make a prediction for each of the 2708 nodes, but only be trained and evaluated on the nodes specified by `idx_train` and `idx_val`. Plot its learning curve and report its validation accuracy. Note that you can run the entire graph through the network in a single training step (unless you run out of memory). My MLP baseline above only uses the nodes that have training and validation labels. That is because it does not look at the neighbors (and their neighbors, ...) of labeled nodes. You will have to use all nodes.

[**Question 2**] Is the MLP baseline equivariant to the permutation of nodes (and the corresponding permutation of the adjacency matrix)? What about the Graph Convolutional Network?

[**Question 3**] Plot the number of training steps (you can choose a few discrete values) vs validation accuracy. Is the MLP baseline or the Graph Convolutional Network more prone to overfitting? Why?

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GCN(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCN, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)

    def forward(self, x, adjacency_matrix):
        device = x.device
        adjacency_matrix = adjacency_matrix + torch.eye(adjacency_matrix.shape[0], device=device)
        adjacency_matrix = adjacency_matrix / adjacency_matrix.sum(dim=1, keepdim=True)
        h = torch.mm(x, self.weight)
        h = torch.matmul(adjacency_matrix, h)
        return h

class GCNClassifier(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, dropout=0.5):
        super(GCNClassifier, self).__init__()
        self.gcn1 = GCN(in_features, hidden_features)
        self.gcn2 = GCN(hidden_features, out_features)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, adjacency_matrix):
        device = x.device
        adjacency_matrix = adjacency_matrix.to(device)
        x = x.to(device)
        h = self.gcn1(x, adjacency_matrix)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.gcn2(h, adjacency_matrix)
        return h


In [ ]:
adj, features, labels, idx_train, idx_val, idx_test = load_data()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
features = features.to(device)
adj = adj.to_dense().to(device)
labels = labels.to(device)
idx_train = idx_train.to(device)
idx_val = idx_val.to(device)

In [ ]:
# Create the GCN model

modelGCN = GCNClassifier(in_features=1433, hidden_features=16, out_features=7).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modelGCN.parameters(), lr=0.01)

In [ ]:
GCN_losses = []
epochs = 300
for epoch in range(epochs):
    optimizer.zero_grad()
    output = modelGCN(features, adj)
    loss = criterion(output[idx_train], labels[idx_train])
    loss.backward()
    optimizer.step()
    GCN_losses.append(loss.item())

    if epoch > 0 and epoch % 50 == 0:
      print("Epoch {:d} / {:d}".format(epoch, epochs))
      print("Mean of last 50 losses: {:f}".format(np.mean(GCN_losses[-50:])))
      
    plt.plot(GCN_losses)
    plt.xlabel("Training step")
    plt.ylabel("Loss")

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(GCN_losses, label="GCN losses")
plt.plot(MLP_losses, label="MLP losses")

plt.legend(loc="upper right")
plt.xlabel("Training step")
plt.ylabel("Loss")

In [ ]:
accuracyGCN = []
accuracyMLP = []

outputs = modelGCN(features,adj)
pred_labels = torch.argmax(outputs[idx_val], dim=-1)
tmp = (labels[idx_val] == pred_labels).float().mean()
accuracyGCN.append(tmp.item())

accuracy = np.mean(accuracyGCN)
print("Correct answer of GCN in {:.1f}% of cases.".format(accuracy * 100))
evaluate(features, labels, idx_val, modelMLP)

In [ ]:
def createList(r1, r2):
    return [item for item in range(r1, r2+1) if item%10==0]
epchos_list = createList(10,1000)


In [ ]:
adj_, features_, labels_, idx_train_, idx_val_, idx_test_ = load_data()

In [ ]:

acclistMLP = []
acclistGCN = []

for epchos in epchos_list:

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  features = features_.to(device)
  adj = adj_.to_dense().to(device)
  labels = labels_.to(device)
  idx_train = idx_train_.to(device)
  idx_val = idx_val_.to(device)

  modelMLP = MLPBaseline().to(device)
  optimizer = optim.Adam(modelMLP.parameters(), lr=0.001)
  MLP_losses = train(features, labels, idx_train, modelMLP, optimizer, epchos)
  acclistMLP.append(evaluate(features, labels, idx_val, modelMLP))

  for epoch in range(epochs):
      optimizer.zero_grad()
      output = modelGCN(features, adj)
      loss = criterion(output[idx_train], labels[idx_train])
      loss.backward()
      optimizer.step()

  accuracyGCN_ = []
  outputs = modelGCN(features,adj)
  pred_labels = torch.argmax(outputs[idx_val], dim=-1)
  tmp = (labels[idx_val] == pred_labels).float().mean()
  accuracyGCN_.append(tmp.item())
  acclistGCN.append(np.mean(accuracyGCN_))

plt.plot(epchos_list,acclistMLP)
plt.plot(epchos_list,acclistGCN)
plt.xlabel("Training step size")
plt.ylabel("Validation Accuracy")

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(epchos_list,acclistMLP,label="MLP")
plt.plot(epchos_list,acclistGCN,label="GCN")
plt.xlabel("Training step size")
plt.ylabel("Validation Accuracy")

plt.legend(loc="upper left")
plt.xlabel("Training step size")
plt.ylabel("Validation Accuracy")
